In [1]:
import pandas as pd
import numpy as np
from urllib.parse import urlparse, unquote
from preprocessor import classify_features

In [2]:
data = pd.read_csv("Full-Dataset/method_na_removed.csv")

In [3]:
data.head(10)

,frame.time,tcp.checksum,Attack_type,http.request.full_uri,tcp.len,tcp.seq,tcp.options,tcp.ack_raw,tcp.ack,tcp.payload,tcp.srcport,GET,POST,TRACE,HTTP/1.0,HTTP/1.1
0,2021 21:35:18.571627000,0x000023bd,SQL_injection,http://192.168.0.128/DVWA/vulnerabilities/sqli...,306.0,1.0,0101080a9b1e676b9eabb8af,1.221191e+09,1.0,474554202f445657412f76756c6e65726162696c697469...,37046.0,True,False,False,False,True
1,2021 21:35:18.642567000,0x0000d49b,SQL_injection,http://192.168.0.128/DVWA/vulnerabilities/sqli...,330.0,1.0,0101080a9b1e67b49eabfa17,2.497556e+09,1.0,474554202f445657412f76756c6e65726162696c697469...,37048.0,True,False,False,False,True
2,2021 21:35:18.691481000,0x00004a68,SQL_injection,http://192.168.0.128/DVWA/vulnerabilities/sqli...,333.0,1.0,0101080a9b1e67e59eabfa5c,1.526554e+09,1.0,474554202f445657412f76756c6e65726162696c697469...,37050.0,True,False,False,False,True
3,2021 21:35:18.816771000,0x00002fad,SQL_injection,http://192.168.0.128/DVWA/vulnerabilities/sqli...,331.0,1.0,0101080a9b1e68629eabfaac,2.083381e+08,1.0,474554202f445657412f76756c6e65726162696c697469...,37052.0,True,False,False,False,True
4,2021 21:35:18.868654000,0x000045cd,SQL_injection,http://192.168.0.128/DVWA/vulnerabilities/sqli...,305.0,1.0,0101080a9b1e68969eabfb0f,4.582203e+08,1.0,474554202f445657412f76756c6e65726162696c697469...,37054.0,True,False,False,False,True
5,2021 21:35:18.928287000,0x0000cb97,SQL_injection,http://192.168.0.128/DVWA/vulnerabilities/sqli...,314.0,1.0,0101080a9b1e68d29eabfb47,3.881938e+09,1.0,474554202f445657412f76756c6e65726162696c697469...,37056.0,True,False,False,False,True
6,2021 21:35:18.980536000,0x0000725d,SQL_injection,http://192.168.0.128/DVWA/vulnerabilities/sqli...,343.0,1.0,0101080a9b1e69069eabfb99,3.203403e+09,1.0,474554202f445657412f76756c6e65726162696c697469...,37058.0,True,False,False,False,True
7,2021 21:35:19.029708000,0x00002a4e,SQL_injection,http://192.168.0.128/DVWA/vulnerabilities/sqli...,337.0,1.0,0101080a9b1e69389eabfbca,1.272756e+08,1.0,474554202f445657412f76756c6e65726162696c697469...,37060.0,True,False,False,False,True
8,2021 21:35:19.083980000,0x000076b2,SQL_injection,http://192.168.0.128/DVWA/vulnerabilities/sqli...,406.0,1.0,0101080a9b1e696e9eabfbfd,3.589361e+09,1.0,474554202f445657412f76756c6e65726162696c697469...,37062.0,True,False,False,False,True
9,2021 21:35:19.130055000,0x0000c7ca,SQL_injection,http://192.168.0.128/DVWA/vulnerabilities/sqli...,439.0,1.0,0101080a9b1e699c9eabfc35,1.710834e+09,1.0,474554202f445657412f76756c6e65726162696c697469...,37064.0,True,False,False,False,True


In [4]:
urlData = pd.Series(
    map(urlparse, data["http.request.full_uri"])
)


In [5]:
def query_params_to_dict(series:pd.Series, to_json=False):
    import json
    return pd.Series([
        (
            json.dumps(dict((ob.split("=")[0], unquote("=".join(ob.split("=")[1:])))for ob in _.split("&")), indent=2, ensure_ascii=True) 
            if to_json else
            dict((ob.split("=")[0], unquote("=".join(ob.split("=")[1:])))for ob in _.split("&")) 
        )
            for _ in series
    ])

In [6]:
urlDatas = pd.DataFrame({
    f"http.request.uri.{key}": [getattr(row,key) for row in urlData]
    for key in ["scheme", "netloc", "path", "params", "query", "fragment",]
})

In [7]:
data = pd.DataFrame({
    **{col: data[col] for col in data.columns if col},
    **{col: urlDatas[col] for col in urlDatas.columns},
})

In [8]:
used_col, unused_col = classify_features(data, ["Attack_type"], debug=True)

scheme : unique value column
netloc : unique value column
params : unique value column
fragment : unique value column


In [9]:
for c in unused_col:
    print(f"{c} = {data[c].unique().shape[0]}")

netloc = 1
fragment = 1
params = 1
scheme = 1


In [10]:
data = data.drop(unused_col, axis="columns")

In [45]:
data["http.request.uri.query"] = query_params_to_dict(data["http.request.uri.query"])

AttributeError: 'dict' object has no attribute 'split'

In [46]:
# split query with index -> Data Regularization
queryArray = []
for i, dict_ in enumerate(data["http.request.uri.query"]):
    for key, value in dict_.items():
        queryArray.append({"original_index":i, "http.request.uri.query.key":key, "http.request.uri.query.value":value})
queryDataFrame = pd.DataFrame(queryArray)

In [47]:
query_droped_data = data.drop(["http.request.uri.query"], axis="columns")
query_droped_data["index"] = range(query_droped_data.shape[0])

In [48]:
queryDataFrame.to_csv("Full-Dataset/processed/queryData.csv", sep=",")
data.drop(["http.request.uri.query"], axis="columns").to_csv("Full-Dataset/processed/url_encoded_query_removed.csv", sep=",")

In [49]:
mergedData = pd.merge(query_droped_data, queryDataFrame, "right", left_on="index", right_on="original_index").drop(["original_index", "http.request.full_uri"], axis="columns")
mergedData.head()

,frame.time,tcp.checksum,Attack_type,http.request.full_uri,tcp.len,tcp.seq,tcp.options,tcp.ack_raw,tcp.ack,tcp.payload,tcp.srcport,GET,POST,TRACE,HTTP/1.0,HTTP/1.1,path,index,query.key,query.value
0,2021 21:35:18.571627000,0x000023bd,SQL_injection,http://192.168.0.128/DVWA/vulnerabilities/sqli...,306.0,1.0,0101080a9b1e676b9eabb8af,1.221191e+09,1.0,474554202f445657412f76756c6e65726162696c697469...,37046.0,True,False,False,False,True,/DVWA/vulnerabilities/sqli_blind/,0,id,2
1,2021 21:35:18.571627000,0x000023bd,SQL_injection,http://192.168.0.128/DVWA/vulnerabilities/sqli...,306.0,1.0,0101080a9b1e676b9eabb8af,1.221191e+09,1.0,474554202f445657412f76756c6e65726162696c697469...,37046.0,True,False,False,False,True,/DVWA/vulnerabilities/sqli_blind/,0,Submit,Submitid=53
2,2021 21:35:18.642567000,0x0000d49b,SQL_injection,http://192.168.0.128/DVWA/vulnerabilities/sqli...,330.0,1.0,0101080a9b1e67b49eabfa17,2.497556e+09,1.0,474554202f445657412f76756c6e65726162696c697469...,37048.0,True,False,False,False,True,/DVWA/vulnerabilities/sqli_blind/,1,id,"2'."")(,.(.("
3,2021 21:35:18.642567000,0x0000d49b,SQL_injection,http://192.168.0.128/DVWA/vulnerabilities/sqli...,330.0,1.0,0101080a9b1e67b49eabfa17,2.497556e+09,1.0,474554202f445657412f76756c6e65726162696c697469...,37048.0,True,False,False,False,True,/DVWA/vulnerabilities/sqli_blind/,1,Submit,Submitid=53
4,2021 21:35:18.691481000,0x00004a68,SQL_injection,http://192.168.0.128/DVWA/vulnerabilities/sqli...,333.0,1.0,0101080a9b1e67e59eabfa5c,1.526554e+09,1.0,474554202f445657412f76756c6e65726162696c697469...,37050.0,True,False,False,False,True,/DVWA/vulnerabilities/sqli_blind/,2,id,"2'FHBrxG<'"">BjxRQk"


In [50]:
mergedData.to_csv("Full-Dataset/processed/merged.csv", sep=",")

In [51]:
col, unused = classify_features(mergedData, ["Attack_type"])
import json
with open("column_data_final.json", "w") as f:
    f.write(json.dumps({"used_columns":col}, indent=4, ensure_ascii=True))

In [11]:
data.head()
# Needing RNN or CNN to process the "query" of the dataset.
# path must be deleted in training, because this dataset's uri.path has the data of "which attack has been used".

,frame.time,http.request.full_uri,tcp.checksum,tcp.options,tcp.payload,Attack_type,GET,POST,TRACE,HTTP/1.0,HTTP/1.1,path,query
0,2021 21:35:18.571627000,http://192.168.0.128/DVWA/vulnerabilities/sqli...,0x000023bd,0101080a9b1e676b9eabb8af,474554202f445657412f76756c6e65726162696c697469...,SQL_injection,True,False,False,False,True,/DVWA/vulnerabilities/sqli_blind/,"{'id': '2', 'Submit': 'Submitid=53'}"
1,2021 21:35:18.642567000,http://192.168.0.128/DVWA/vulnerabilities/sqli...,0x0000d49b,0101080a9b1e67b49eabfa17,474554202f445657412f76756c6e65726162696c697469...,SQL_injection,True,False,False,False,True,/DVWA/vulnerabilities/sqli_blind/,"{'id': '2'."")(,.(.(', 'Submit': 'Submitid=53'}"
2,2021 21:35:18.691481000,http://192.168.0.128/DVWA/vulnerabilities/sqli...,0x00004a68,0101080a9b1e67e59eabfa5c,474554202f445657412f76756c6e65726162696c697469...,SQL_injection,True,False,False,False,True,/DVWA/vulnerabilities/sqli_blind/,"{'id': '2'FHBrxG<'"">BjxRQk', 'Submit': 'Submit..."
3,2021 21:35:18.816771000,http://192.168.0.128/DVWA/vulnerabilities/sqli...,0x00002fad,0101080a9b1e68629eabfaac,474554202f445657412f76756c6e65726162696c697469...,SQL_injection,True,False,False,False,True,/DVWA/vulnerabilities/sqli_blind/,"{'id': '2) AND 9522=9017 AND (5824=5824', 'Sub..."
4,2021 21:35:18.868654000,http://192.168.0.128/DVWA/vulnerabilities/sqli...,0x000045cd,0101080a9b1e68969eabfb0f,474554202f445657412f76756c6e65726162696c697469...,SQL_injection,True,False,False,False,True,/DVWA/vulnerabilities/sqli_blind/,"{'id': '2 AND 5107=2126', 'Submit': 'Submitid=..."


In [28]:
data.to_csv("Full-Dataset/processed/url_encoded_dataset.csv", sep=",")